# Construcción de features para el S&P 500

## Procedimientos

- Descargar datos:
    - Serie S&P 500
    - Serie para las compañias
    - Series para las monedas
    - Series para los indices
- Calcular retornos
- Calcular variable de clase
- Calcular desfase temporal del retorno
- Calcular indicadores técnicos
- Cargar indicadores económicos
- Guardar features y variable de clase

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pickle as pk
import pandas as pd
from pandas_datareader import data
import fix_yahoo_finance

fix_yahoo_finance.pdr_override()

from datetime import datetime

from pyCBT.providers.scrappers import slickcharts, wikipedia
from pyCBT.common.files import exist
from talib import RSI, CCI, ROC, MOM, WILLR

from pyCBT.providers.oanda import account, historical

oanda_client = account.Client()

In [3]:
sp500_slick = slickcharts.get_sp500_metadata()
sp500_slick.set_index(keys="Symbol", inplace=True)
sp500_slick.sort_index(inplace=True)

sp500_wiki = wikipedia.get_sp500_metadata()
sp500_wiki.set_index(keys="Symbol", inplace=True)
sp500_wiki.sort_index(inplace=True)
sp500_wiki.drop(columns="Company", inplace=True)

sp500_md = pd.concat((sp500_wiki, sp500_slick), axis=1)
sp500_md.sort_values(by="Weight")

,Sector,Company,Weight
Symbol,,,
AAPL,Information Technology,Apple Inc.,1.0
MSFT,Information Technology,Microsoft Corporation,2.0
AMZN,Consumer Discretionary,Amazon.com Inc.,3.0
FB,Information Technology,Facebook Inc. Class A,4.0
BRK.B,Financials,Berkshire Hathaway Inc. Class B,5.0
JPM,Financials,JPMorgan Chase & Co.,6.0
JNJ,Health Care,Johnson & Johnson,7.0
XOM,Energy,Exxon Mobil Corporation,8.0
GOOG,Information Technology,Alphabet Inc. Class C,9.0


In [4]:
# definir fechas
from_date, to_date = datetime(2005, 1, 1), datetime.today().date()
# descargar datos
# TODO: use alpha-vantage or OANDA instead
sectors_stooq = {
    "consumer-discretionary": "BI.F",
    "consumer-staples": "BL.F",
    "energy": "BM.F",
    "financials": "BN.F",
    "health-care": "BS.F",
    "industrials": "JG.F",
    "information-technology": "JB.F",
    "materials": "JA.F",
    "utilities": "JE.F",
    "real-state": "BK.F"
}
indices_yahoo = {
    "DJI": "^DJI",
    "HSI": "^HSI",
    "FCHI": "^FCHI",
    "NYA": "^NYA",
    "VIX": "^VIX"
}
# TODO: use alpha-vantage or OANDA instead
indices_stooq = {
    "DXY": "USD_I",
    "FTSE": "X.F",
    "DAX": "^DAX",
    "DJA": "DIA.US",
    "NASDAQ": "^NDX",
    "N225": "^NKX",
    "SSE": "^SHC",
    "VXN": "R3.C",
    "VXO": "VI.F"
}
# TODO: use alpha-vantage instead
stocks_stooq = {
    "AAPL": "AAPL.US",
    "MSFT": "MSFT.US",
    "AMZN": "AMZN.US",
    "FB": "FB.US",
    "JPM": "JPM.US",
    "BRK.B": "BRK-B.US",
    "JNJ": "JNJ.US",
    "GOOG": "GOOG.US",
    "XOM": "XOM.US"
}
# TODO: use OANDA instead
currencies_yahoo = {
    "USDCAD": "CAD=X",
    "GBPUSD": "GBPUSD=X",
    "USDCNY": "CNY=X",
    "USDJPY": "JPY=X",
    "EURUSD": "EURUSD=X"
}
cfds_oanda = {
    "Crude Oil": "WTICO_USD",
    "Gold": "XAU_USD"
}
if exist("../data/pickles/filter-sp500-datasets.p"):
    sp500, sp500_sectors, major_stocks, major_indices, major_currencies, major_cfds, major_econ = pk.load(open("../data/pickles/filter-sp500-datasets.p", "rb"))
else:
    major_econ = pd.read_csv("../data/processed/filter-analysis/economic-indicators-raw.csv", index_col="Date").sort_index()
    
    sp500 = data.get_data_stooq("^SPX").sort_index()

    sp500_sectors, major_stocks, major_indices, major_currencies, major_cfds = {}, {}, {}, {}, {}
    for ticker in indices_yahoo:
        major_indices[ticker] = data.get_data_yahoo(indices_yahoo[ticker], from_date, to_date).sort_index()
    for ticker in indices_stooq:
        major_indices[ticker] = data.get_data_stooq(indices_stooq[ticker]).sort_index()
    for ticker in stocks_stooq:
        major_stocks[ticker] = data.get_data_stooq(stocks_stooq[ticker]).sort_index()
    for ticker in currencies_yahoo:
        major_currencies[ticker] = data.get_data_yahoo(currencies_yahoo[ticker], from_date, to_date).sort_index()
    for sector in sectors_stooq:
        sp500_sectors[sector] = data.get_data_stooq(sectors_stooq[sector]).sort_index()

    for ticker in cfds_oanda:
        candles = historical.Candles(
            client=oanda_client,
            instrument=cfds_oanda[ticker],
            resolution="D",
            from_date=from_date.strftime("%Y-%m-%d"),
            to_date=to_date.strftime("%Y-%m-%d"),
            datetime_fmt="%Y-%m-%d",
            timezone="America/New_York"
        )
        major_cfds[ticker] = candles.as_dataframe()
        major_cfds[ticker].index.name = "Date"
        major_cfds[ticker].index = pd.to_datetime(major_cfds[ticker].index)
        
    pk.dump((sp500, sp500_sectors, major_stocks, major_indices, major_currencies, major_cfds, major_econ), open("../data/pickles/filter-sp500-datasets.p", "wb"))

In [5]:
# construir tabla master
X_master = pd.DataFrame(index=sp500.index, data=None)
for d in [sp500_sectors, major_stocks, major_indices, major_currencies, major_cfds]:
    for ticker in sorted(d):
        X_master[ticker] = d[ticker].Close.shift()
# filtrar NANs
X_master = X_master.dropna(how="all", axis=0)
sp500 = sp500.filter(items=X_master.index, axis=0)
# calcular retornos
Y_return = sp500.Close.pct_change()
X_return = X_master.pct_change()
# calcular indicadores técnicos: rsi, williams_r, cci_symbol, roc, mom
X_techin = pd.DataFrame(index=sp500.index, data=None)
X_techin["RSI"] = RSI(sp500.Close.shift(), 5)
X_techin["%R"] = WILLR(sp500.High.shift(), sp500.Low.shift(), sp500.Close.shift(), 5)
X_techin["CCI"] = CCI(sp500.High.shift(), sp500.Low.shift(), sp500.Close.shift(), 5)
X_techin["ROC"] = ROC(sp500.Close.shift(), 5)
X_techin["MOM"] = MOM(sp500.Close.shift(), 5)
# concatenar retornos + indicadores técnicos + indicadores económicos
# TODO: ordenar columnas por tipo de indicador
X_raw = pd.concat((X_return, X_techin), axis="columns")
Y_raw = pd.DataFrame(
    index=sp500.index,
    data={
        "Price": sp500.Close,
        "Return": Y_return,
        "Class": (Y_return>0.0).astype(int),
    }
)

In [9]:
X_raw.tail()

,consumer-discretionary,consumer-staples,energy,financials,health-care,industrials,information-technology,materials,real-state,utilities,...,USDCAD,USDCNY,USDJPY,Crude Oil,GOld,RSI,%R,CCI,ROC,MOM
Date,,,,,,,,,,,,,,,,,,,,,
2018-04-25,-0.015830,-0.005535,-0.010645,-0.006999,-0.007742,-0.028031,-0.017798,-0.027010,0.001373,0.006715,...,-0.001767,-0.001615,0.000386,0.004014,-0.006455,30.331220,-82.789258,-122.908613,-2.654089,-71.83
2018-04-26,0.003607,0.000596,0.007491,-0.001322,0.004632,0.002449,-0.000614,0.004792,-0.003428,-0.000785,...,0.001147,0.002981,0.005156,0.002411,-0.003932,34.302713,-70.355994,-105.086877,-2.556264,-69.24
2018-04-27,0.016416,0.006555,0.015412,0.000441,0.010921,-0.003935,0.019053,0.006413,0.012384,0.010799,...,NaN,NaN,NaN,NaN,NaN,53.255548,-33.222591,28.164568,-0.972474,-26.19
2018-04-30,0.004874,0.005131,-0.010385,0.002352,0.005522,-0.001499,-0.001960,-0.002614,0.013252,0.010490,...,0.000296,0.001945,-0.002432,0.005866,-0.000819,55.005355,-19.243792,61.362495,-0.008614,-0.23
2018-05-01,-0.003519,-0.006479,-0.000942,-0.009824,-0.015399,-0.013508,-0.006043,-0.013925,-0.004359,-0.003268,...,-0.000654,-0.000773,0.002062,-0.015774,-0.008947,40.914412,-50.084650,31.761964,-0.832868,-22.24


In [10]:
Y_raw.tail()

,Class,Price,Return
Date,,,
2018-04-25,1,2639.40,0.001837
2018-04-26,1,2666.94,0.010434
2018-04-27,1,2669.91,0.001114
2018-04-30,0,2648.05,-0.008188
2018-05-01,1,2654.80,0.002549


In [8]:
# guardar tablas crudas
X_raw.reset_index().to_csv("../data/processed/sp500-ml/features-raw.csv", index=False)
Y_raw.reset_index().to_csv("../data/processed/sp500-ml/target-raw.csv", index=False)

# guardar tablas de los últimos ~10 años
X_short = X_raw.dropna(how="all", axis=0)
X_short = X_short[X_short.index>"2009-01-01"]
Y_short = Y_raw.filter(items=X_short.index)
X_short.reset_index().to_csv("../data/processed/sp500-ml/features.csv", index=False)
Y_short.reset_index().to_csv("../data/processed/sp500-ml/target.csv", index=False)